In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
import numpy as np

# Define dataset path
dataset_path = "/kaggle/input/face-detection-by-kazmir/Face detection"

# Load dataset with data augmentation
batch_size = 16
img_size = (224, 224)

# Load the dataset before augmentation to retrieve class names
train_ds, val_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

# Check the number of classes by inspecting the class names
class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Classes found: {class_names}")

# Now apply data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

# Apply the augmentation to the dataset
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

# Use the MobileNetV2 pre-trained model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model to use it as a feature extractor
base_model.trainable = False

# Add classification layers on top with dropout to prevent overfitting
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling layer
x = Dense(128, activation='relu')(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Add dropout layer with 50% dropout rate
predictions = Dense(num_classes, activation='softmax')(x)  # Number of classes

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 20  # Increased the number of epochs
history = model.fit(train_ds, epochs=epochs)

# Save the model
model.save('/kaggle/working/face_detection_model_with_augmentation.h5')

# Save the TensorFlow Lite model
with open('/kaggle/working/face_detection_model_with_augmentation.tflite', 'wb') as f:
    f.write(tflite_model)


Found 25 files belonging to 5 classes.
Using 20 files for training.
Using 5 files for validation.
Classes found: ['Kazmir', 'Mudassir', 'ashik', 'riyan', 'sayed']
Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 104ms/step - accuracy: 0.2167 - loss: 2.5849
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - accuracy: 0.3375 - loss: 1.9081
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.3042 - loss: 1.2626
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.5000 - loss: 1.3317
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.4250 - loss: 1.3687
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.5750 - loss: 1.2017
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.5750 - loss: 1.2660
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - accuracy: 0.7500 - loss: 0.5391
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.7292 - loss: 0.6287
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.5542 - loss: 0.7997
Epoch